# COURSERA CAPSTONE PROJECT

## Battle of the Neighboorhoods

### Section Alpha: Introduction


Description of Problem
    
    A couple has decided to relocate within the United States. They have created a list of requirements.  This list includes a population range of under 100000 and a few community amenities, such as a grocery store, a library, a post office, a gym or outdoor recreation, and restaurants.  I am tasked with determining the cities and towns which fall within this desirable range.  

In [1]:
# import libraries
import numpy as np # for vectorized data
import pandas as pd # for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # for JSON files
import requests # for requests
from pandas.io.json import json_normalize # to tranform JSON file into a pandas dataframe

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # to convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # for map rendering
from folium import plugins

print('LIBRARIES IMPORTED.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

### Section Beta:  Data


Description of Data

    Data will be aquired from the U.S. Census Bureau to idendify communities withing the desired population range.  These metropolitan and micropolitan areas will then be passed through the Google Maps API in order to geocode each loaction by latitude and longintude.  These coordiates will then be used in the Foursquare API, along with the venue categories selected by the couple in order, to aquire information regarding community amenities for each location.  Lastly, these will be mapped to aid in visualization.  

This product uses the Census Bureau Data API but is not endorsed or certified by the Census Bureau.

In [2]:
#Census Bureau API Key and Call
CensusBureau_Key = ''
CensusBureau_url='https://api.census.gov/data/2018/pep/population?get=GEONAME,POP&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key={}'.format(CensusBureau_Key)
#create dataframe
df = pd.read_json(CensusBureau_url)
df.head()

,0,1,2
0,GEONAME,POP,metropolitan statistical area/micropolitan sta...
1,"Aberdeen, SD Micro Area",43191,10100
2,"Aberdeen, WA Micro Area",73901,10140
3,"Abilene, TX Metro Area",171451,10180
4,"Ada, OK Micro Area",38247,10220


In [3]:
df = df.drop(columns =[2])
df.columns = ["CITY","POPULATION"]
df = df.drop(0)
df.head()

,CITY,POPULATION
1,"Aberdeen, SD Micro Area",43191
2,"Aberdeen, WA Micro Area",73901
3,"Abilene, TX Metro Area",171451
4,"Ada, OK Micro Area",38247
5,"Adjuntas, PR Micro Area",17302


In [4]:
df.dtypes

CITY          object
POPULATION    object
dtype: object

In [5]:
df['POPULATION']=df['POPULATION'].astype(str).astype(int)
df['CITY']=df['CITY'].astype(str)
df.dtypes

CITY          object
POPULATION     int64
dtype: object

In [6]:
df=df[df['POPULATION']<=100000]
df['CITY']=df['CITY'].str.rstrip(' Area')
df['CITY']=df['CITY'].str.rstrip(' Micro')
df['CITY']=df['CITY'].str.rstrip(' Metro')
df.reset_index(drop=True)
df.head()

,CITY,POPULATION
1,"Aberdeen, SD",43191
2,"Aberdeen, WA",73901
4,"Ada, OK",38247
5,"Adjuntas, PR",17302
6,"Adrian, MI",98266


In [7]:
df.shape

(559, 2)

In [8]:
GoogleMaps_Key = '' # Google Maps API Key 
#create Google Maps API request url and append to dataframe
lats=[] #create empty latitude list
lngs=[] #create empty longitude list
for i in df['CITY']:
    try:
        GoogleMaps_url='https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(GoogleMaps_Key,i)
        response=requests.get(GoogleMaps_url).json()
        coordinates=response['results'][0]['geometry']['location']
        lats.append(coordinates['lat'])
        lngs.append(coordinates['lng'])
    except:
        pass
df2=pd.DataFrame()
df2['Latitude']=lats
df2['Longitude']=lngs
df2.reset_index(drop=True)
df2.head()
#df['Latitude']=lats
#df['Longitude']=lngs
#df.reset_index(drop=True)

,Latitude,Longitude
0,45.464698,-98.486483
1,46.975371,-123.815722
2,34.774531,-96.678345
3,18.163485,-66.723158
4,41.897547,-84.037166


In [9]:
df2.shape

(558, 2)

In [10]:
df=df.iloc[0:558]
df.shape

(558, 2)

In [11]:
df.to_csv(r'F:\dataframe.csv')
df2.to_csv(r'F:\dataframe2.csv')

In [12]:
data=pd.concat([df, df2], axis = 1)
data.head()

,CITY,POPULATION,Latitude,Longitude
0,NaN,NaN,45.464698,-98.486483
1,"Aberdeen, SD",43191.0,46.975371,-123.815722
2,"Aberdeen, WA",73901.0,34.774531,-96.678345
3,NaN,NaN,18.163485,-66.723158
4,"Ada, OK",38247.0,41.897547,-84.037166


In [13]:
#create category list
#Grocery Store 4bf58dd8d48988d118951735
#Food 4d4b7105d754a06374d81259
#Library 4bf58dd8d48988d12f941735
#Outdoors & Recreations 4d4b7105d754a06377d81259
#Post Office 4bf58dd8d48988d172941735

CategoryIds=['4bf58dd8d48988d118951735','4bf58dd8d48988d12f941735','4d4b7105d754a06377d81259','4bf58dd8d48988d172941735']
categoryId=','.join(CategoryIds)
print(categoryId)

4bf58dd8d48988d118951735,4bf58dd8d48988d12f941735,4d4b7105d754a06377d81259,4bf58dd8d48988d172941735


In [14]:
client_id = '' # Foursquare API ID
client_secret = '' # Foursquare API Secret
version = '20190810' # Foursquare API version YYYYMMDD
radius = 50 # define radius
limit = 100 # limit the number of venues returned by Foursquare API
# create URL

venues_list=[]
for lat, lon, in zip(data['Latitude'],data['Longitude']):
    FourSquare_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}'.format(
        client_id,
        client_secret, 
        version, 
        lat,
        lon,
        radius, 
        categoryId,
        limit)
        
    results2=requests.get(FourSquare_url).json()['response']['groups'][0]['items']
    
    for venue in results2:
        venues_list.append((
            lat,
            lon,
            venue['venue']['name'],
            venue['venue']['categories'][0]['name']))
            
venues_df=pd.DataFrame(venues_list)
venues_df.columns=['City','Population','Latitude','Longitude','Venue Name','Venue Type']

print(venues_df.shape)
venues_df.head()

KeyError: 'groups'

In [21]:
venues_df.to_csv(r'F:\dataframevenues.csv')

In [22]:
venues_df.groupby('Latitude')['Latitude'].count().sort_values(ascending=False)
venues_df.head()

,Latitude,Longitude,Venue Name,Venue Type
0,43.378920,-84.659727,L.E.A.N.,Gym / Fitness Center
1,34.637765,-99.334044,Gym Virtual,Athletics & Sports
2,32.072386,-84.232688,"Caravelle Boat Group, LLC.",Harbor / Marina
3,32.072386,-84.232688,Workout Joe's,Gym / Fitness Center
4,41.634773,-84.999409,Fitt4life,Gym


In [23]:
venues_df_top=venues_df.iloc[0:25]

In [25]:
map = folium.Map(location=[39.8, -98.6], zoom_start=10)
venues_df_top.apply(lambda row:folium.Marker(location=[row["Latitude"], 
                                                  row["Longitude"]]).add_to(map),
     axis=1)
map

In [26]:
venues_df.head(25)

,Latitude,Longitude,Venue Name,Venue Type
0,43.378920,-84.659727,L.E.A.N.,Gym / Fitness Center
1,34.637765,-99.334044,Gym Virtual,Athletics & Sports
2,32.072386,-84.232688,"Caravelle Boat Group, LLC.",Harbor / Marina
3,32.072386,-84.232688,Workout Joe's,Gym / Fitness Center
4,41.634773,-84.999409,Fitt4life,Gym
5,34.174261,-97.143625,Deaver Lawn Care,Garden
6,34.174261,-97.143625,The Pumpkin Patch at Woodbine Farms,Farm
7,40.868668,-82.318218,Family Pools & Spas,Pool
8,40.868668,-82.318218,The Corner Park.,Park
9,46.187884,-123.831253,Garden of Surging Waves,Park
